In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import os

import torchvision
import torchvision.transforms as transforms

from model.CNN import *
from model.VGG import *
from model.ResNet import *

## Is CUDA available?

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]= "0"

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
device

device(type='cuda')

## Dataset

In [3]:
transform = transforms.Compose(
    [transforms.RandomCrop(32, padding=4),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=512,
                                          shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                          shuffle=False)

60.8%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

75.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100.0%

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## Model

In [4]:
# model = CNN().to(device)
model = VGG11().to(device)      # VGG11, VGG13, VGG16, VGG19
# model = ResNet18().to(device)   # ResNet18, ResNet34, ResNet50, ResNet101, ResNet152


# model = nn.DataParallel(model)

In [5]:
epochs = 100
learning_rate = 5e-3
momentum = 0.9

optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=5e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
loss_function = nn.CrossEntropyLoss()

## Train

In [6]:
def train_process(epoch):
    model.train()

    train_loss = 0
    train_correct = 0

    for idx, (images, labels) in enumerate(trainloader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = loss_function(outputs, labels)
        train_loss += loss
        loss.backward()
        optimizer.step()

        pred = outputs.argmax(dim=1, keepdims=True)
        train_correct += pred.eq(labels.view_as(pred)).sum().item()

        if idx % 20 == 0:
            print('Train Epoch : {} [{}/{} ({:.0f}%)]\tLoss : {:.6f}'.format(
                epoch, idx * len(images), len(trainloader.dataset),
                       100 * idx / len(trainloader), loss.item()
            ))

    scheduler.step()

    train_loss /= idx

    print('Train set - Average Loss : {:.4f}, Accuracy : {}/{} ({:.2f}%)'.format(
        train_loss, train_correct, len(trainloader.dataset), 100 * train_correct / len(trainloader.dataset)))


## Test

In [7]:
def test_process():
    model.eval()

    test_loss = 0
    test_correct = 0

    with torch.no_grad():
        for idx, (images, labels) in enumerate(testloader):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            test_loss += loss_function(outputs, labels)

            pred = outputs.argmax(dim=1, keepdims=True)
            test_correct += pred.eq(labels.view_as(pred)).sum().item()

    test_loss /= idx

    print('Test set - Average Loss : {:.4f}, Accuracy : {}/{} ({:.2f}%)'.format(
        test_loss, test_correct, len(testloader.dataset), 100 * test_correct / len(testloader.dataset)))


## Let`s Train

In [8]:
for epoch in range(1, epochs + 1):
    train_process(epoch)
    test_process()

Train Epoch : 1 [0/50000 (0%)]	Loss : 2.440488
Train Epoch : 1 [10240/50000 (20%)]	Loss : 1.579801
Train Epoch : 1 [20480/50000 (41%)]	Loss : 1.396524
Train Epoch : 1 [30720/50000 (61%)]	Loss : 1.296080
Train Epoch : 1 [40960/50000 (82%)]	Loss : 1.194924
Train set - Average Loss : 1.4605, Accuracy : 23515/50000 (47.03%)
Test set - Average Loss : 1.4554, Accuracy : 4841/10000 (48.41%)
Train Epoch : 2 [0/50000 (0%)]	Loss : 1.128043
Train Epoch : 2 [10240/50000 (20%)]	Loss : 1.059035
Train Epoch : 2 [20480/50000 (41%)]	Loss : 0.995458
Train Epoch : 2 [30720/50000 (61%)]	Loss : 0.965640
Train Epoch : 2 [40960/50000 (82%)]	Loss : 0.956565
Train set - Average Loss : 1.0525, Accuracy : 31268/50000 (62.54%)
Test set - Average Loss : 1.1085, Accuracy : 6127/10000 (61.27%)
Train Epoch : 3 [0/50000 (0%)]	Loss : 0.925336
Train Epoch : 3 [10240/50000 (20%)]	Loss : 0.933658
Train Epoch : 3 [20480/50000 (41%)]	Loss : 0.863727
Train Epoch : 3 [30720/50000 (61%)]	Loss : 0.944808
Train Epoch : 3 [40960/

Train Epoch : 22 [10240/50000 (20%)]	Loss : 0.264494
Train Epoch : 22 [20480/50000 (41%)]	Loss : 0.259282
Train Epoch : 22 [30720/50000 (61%)]	Loss : 0.281807
Train Epoch : 22 [40960/50000 (82%)]	Loss : 0.251011
Train set - Average Loss : 0.2754, Accuracy : 45206/50000 (90.41%)
Test set - Average Loss : 0.5614, Accuracy : 8232/10000 (82.32%)
Train Epoch : 23 [0/50000 (0%)]	Loss : 0.255302
Train Epoch : 23 [10240/50000 (20%)]	Loss : 0.211146
Train Epoch : 23 [20480/50000 (41%)]	Loss : 0.251862
Train Epoch : 23 [30720/50000 (61%)]	Loss : 0.256501
Train Epoch : 23 [40960/50000 (82%)]	Loss : 0.266558
Train set - Average Loss : 0.2624, Accuracy : 45417/50000 (90.83%)
Test set - Average Loss : 0.6135, Accuracy : 8054/10000 (80.54%)
Train Epoch : 24 [0/50000 (0%)]	Loss : 0.273066
Train Epoch : 24 [10240/50000 (20%)]	Loss : 0.223962
Train Epoch : 24 [20480/50000 (41%)]	Loss : 0.238611
Train Epoch : 24 [30720/50000 (61%)]	Loss : 0.265410
Train Epoch : 24 [40960/50000 (82%)]	Loss : 0.247170
Trai

Train Epoch : 43 [10240/50000 (20%)]	Loss : 0.097200
Train Epoch : 43 [20480/50000 (41%)]	Loss : 0.051936
Train Epoch : 43 [30720/50000 (61%)]	Loss : 0.073433
Train Epoch : 43 [40960/50000 (82%)]	Loss : 0.083275
Train set - Average Loss : 0.0749, Accuracy : 48830/50000 (97.66%)
Test set - Average Loss : 0.4507, Accuracy : 8710/10000 (87.10%)
Train Epoch : 44 [0/50000 (0%)]	Loss : 0.081604
Train Epoch : 44 [10240/50000 (20%)]	Loss : 0.070228
Train Epoch : 44 [20480/50000 (41%)]	Loss : 0.085557
Train Epoch : 44 [30720/50000 (61%)]	Loss : 0.068195
Train Epoch : 44 [40960/50000 (82%)]	Loss : 0.059419
Train set - Average Loss : 0.0719, Accuracy : 48929/50000 (97.86%)
Test set - Average Loss : 0.4588, Accuracy : 8680/10000 (86.80%)
Train Epoch : 45 [0/50000 (0%)]	Loss : 0.089316
Train Epoch : 45 [10240/50000 (20%)]	Loss : 0.083462
Train Epoch : 45 [20480/50000 (41%)]	Loss : 0.079215
Train Epoch : 45 [30720/50000 (61%)]	Loss : 0.050921
Train Epoch : 45 [40960/50000 (82%)]	Loss : 0.077575
Trai

Train Epoch : 64 [10240/50000 (20%)]	Loss : 0.034695
Train Epoch : 64 [20480/50000 (41%)]	Loss : 0.057044
Train Epoch : 64 [30720/50000 (61%)]	Loss : 0.055451
Train Epoch : 64 [40960/50000 (82%)]	Loss : 0.035937
Train set - Average Loss : 0.0482, Accuracy : 49352/50000 (98.70%)
Test set - Average Loss : 0.4860, Accuracy : 8679/10000 (86.79%)
Train Epoch : 65 [0/50000 (0%)]	Loss : 0.038491
Train Epoch : 65 [10240/50000 (20%)]	Loss : 0.043314
Train Epoch : 65 [20480/50000 (41%)]	Loss : 0.055738
Train Epoch : 65 [30720/50000 (61%)]	Loss : 0.057350
Train Epoch : 65 [40960/50000 (82%)]	Loss : 0.055896
Train set - Average Loss : 0.0470, Accuracy : 49319/50000 (98.64%)
Test set - Average Loss : 0.4794, Accuracy : 8728/10000 (87.28%)
Train Epoch : 66 [0/50000 (0%)]	Loss : 0.056007
Train Epoch : 66 [10240/50000 (20%)]	Loss : 0.047218
Train Epoch : 66 [20480/50000 (41%)]	Loss : 0.055297
Train Epoch : 66 [30720/50000 (61%)]	Loss : 0.037783
Train Epoch : 66 [40960/50000 (82%)]	Loss : 0.056206
Trai

Train Epoch : 85 [10240/50000 (20%)]	Loss : 0.041308
Train Epoch : 85 [20480/50000 (41%)]	Loss : 0.045599
Train Epoch : 85 [30720/50000 (61%)]	Loss : 0.051093
Train Epoch : 85 [40960/50000 (82%)]	Loss : 0.034675
Train set - Average Loss : 0.0461, Accuracy : 49342/50000 (98.68%)
Test set - Average Loss : 0.4823, Accuracy : 8674/10000 (86.74%)
Train Epoch : 86 [0/50000 (0%)]	Loss : 0.053070
Train Epoch : 86 [10240/50000 (20%)]	Loss : 0.047655
Train Epoch : 86 [20480/50000 (41%)]	Loss : 0.049203
Train Epoch : 86 [30720/50000 (61%)]	Loss : 0.032434
Train Epoch : 86 [40960/50000 (82%)]	Loss : 0.032057
Train set - Average Loss : 0.0435, Accuracy : 49398/50000 (98.80%)
Test set - Average Loss : 0.4821, Accuracy : 8689/10000 (86.89%)
Train Epoch : 87 [0/50000 (0%)]	Loss : 0.049750
Train Epoch : 87 [10240/50000 (20%)]	Loss : 0.040525
Train Epoch : 87 [20480/50000 (41%)]	Loss : 0.054634
Train Epoch : 87 [30720/50000 (61%)]	Loss : 0.029740
Train Epoch : 87 [40960/50000 (82%)]	Loss : 0.048211
Trai